Note: This notebook is being run by the presenter locally and will fail on Binder.

# Distributed
As we covered at the beginning Dask has the ability to run work on mulitple machines using the distributed scheduler.

Until now we have actually been using the distributed scheduler for our work, but just on a single machine.

When we instantiate a Client() object with no arguments it will attempt to locate a Dask cluster. It will check your local Dask config and environment variables to see if connection information has been specified. If not it will create an instance of LocalCluster and use that.

Specifying connection information in config is useful for system administrators to provide access to their users. We do this in the Dask Helm Chart for Kubernetes, the chart installs a multi-node Dask cluster and a Jupyter server on a Kubernetes cluster and Jupyter is preconfigured to discover the distributed cluster.

### Remote clusters via SLURM
A common way to distribute your work onto multiple machines is via an HPC System with SLURM. Dask has a cluster manager which handles nodes called `SLURMCluster`.

In [ ]:
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

You need to specify information about the nodes that you would like to use. Below, we specify information for the `blanca-lundquist` nodes on Blanca.

In [ ]:
### Set up Cluster (for Blanca)
## Node parameters
##     I obtained this info from `scontrol show nodes`
PARTITION = 'blanca-lundquist'  # aka "queue"
ACCOUNT = 'blanca-lundquist'    # aka "project"
NUMCORES = 40                   # Total number of cores
TOTMEM = '100GB'    # Total memory; `FreeMem=100GB`, `RealMemory=185GB`
# INTERFACE = 'ib0'
WALLTIME = '12:00:00'
TMPDIR = '/rc_scratch/olry2348/tmp'
NUMPROCS = 1

cluster = SLURMCluster(
    queue=PARTITION,
    project=ACCOUNT,
    cores=NUMCORES,
    processes=NUMPROCS,
    memory=TOTMEM,
    walltime=WALLTIME,
    local_directory=TMPDIR)#,
#     interface=INTERFACE)

NUMNODES = 3
cluster.scale(jobs=NUMNODES)

In [ ]:
client = Client(cluster)
client